fork元のサンプルコード https://www.kaggle.com/ruruamour/simple-sample-code では良い精度が出ませんでした。 
  
精度が出なかった要因の１つに、学習データのバリエーションが不足していたことが考えられます。

  
このNotebookでは、画像のaugmentation（水増し）を行い、学習データのバリエーションを増やす方法を書きます。


## 1. 準備

In [ ]:
import pandas as pd
root_dir = "/kaggle/input/mj1-anomaly-images-detection-challenge/"
train_csv_filepath = root_dir + "train.csv"

# ファイルの読み込み
train_df = pd.read_csv(train_csv_filepath)

In [ ]:
resize_w = 256
resize_h = 256
channel = 3

import cv2
# 画像が大きいと計算が遅いため、リサイズ縮小
def resize(tmp_image):
    return cv2.resize(tmp_image , (resize_h, resize_w))

# 4次元配列化()　
def to_4d(tmp_image):
    return tmp_image.reshape(1, resize_h, resize_w, channel)
    

# 256段階の色調を0.0~1.0にする
def normalize(tmp_image):
    return tmp_image / 255.0

# 画像の前処理付きロード
def load_preprocessed_image(image_filepath):
    tmp_image = cv2.imread(image_filepath)
    tmp_image = resize(tmp_image)
    tmp_image = normalize(tmp_image)
    tmp_image = to_4d(tmp_image)

    return tmp_image

In [ ]:
import numpy as np
from keras.utils import np_utils

images = None
for fn in train_df['filename']:
    image_filepath = root_dir + 'train/' + fn
    tmp_image = load_preprocessed_image(image_filepath)
    if (images is None):
        images = tmp_image
    else:
        images = np.vstack((images, tmp_image))

anomaly_flags = np.array([flag for flag in train_df['anomaly']])
anomaly_flags = np_utils.to_categorical(anomaly_flags, 2)


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.333, random_state=0)

for train_index, test_index in sss.split(images, anomaly_flags):
    X_train = images[train_index]
    y_train = anomaly_flags[train_index]
    X_test = images[test_index]
    y_test = anomaly_flags[test_index]

In [ ]:
# over-samplingを試します。

tmp = pd.DataFrame(y_train[:, 1]).value_counts().values
print(tmp)
label_ok_num = tmp[0]
label_ng_num = tmp[1]

while(label_ok_num != label_ng_num):
    rand_index = np.random.randint(0, len(y_train))

    label_is_ng = (y_train[rand_index, 1] == 1.0)
    if label_is_ng:
        X_train = np.vstack((X_train, [X_train[rand_index]]))
        y_train = np.vstack((y_train, [y_train[rand_index]]))
        label_ng_num += 1
    print(label_ng_num, end='\r')

# 2. 画像を表示してみる

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

for i, f in enumerate(train_df['filename'][:5]):
    plt.subplot(1,5,i+1)
    image_filepath = root_dir + 'train/' + f
    tmp_image = cv2.imread(image_filepath)
    plt.imshow(cv2.cvtColor(tmp_image, cv2.COLOR_BGR2RGB)) # OpenCV は色がGBR順なのでRGB順に並べ替える

いくつか試しに表示してみましたが、  
釘の向きがバラバラでした。

# 3. ImageDataGeneratorで画像の水増しを行う

kerasのImageDataGeneratorを使って、学習のたびにランダムな画像処理を行って学習させるようにします。  
今回は釘の向きがバラバラであったので、画像の回転を行うことにします。

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=360)

datagen.fit(X_train)

In [ ]:
for i, f in enumerate(datagen.flow(X_train, y_train, batch_size=1)):
    tmp_image = f[0][0]*256
    tmp_image = tmp_image.astype(np.uint8)
    print(tmp_image.shape)
    plt.imshow(tmp_image) # OpenCV は色がGBR順なのでRGB順に並べ替える
    if (i>=4):
        break

# 3.学習モデルの作成

In [ ]:
import numpy as np
#データの読み込みと前処理

from keras.datasets import mnist
#kerasでCNN構築
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam

 
'''
CNNの構築
'''
def cnn_model():
    model = Sequential()

    model.add(Conv2D(filters=10, kernel_size=(4,4), padding='same', input_shape=(256, 256, 3), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(3,3), padding='same', input_shape=(64, 64, 8), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(2,2), padding='same', input_shape=(16, 16, 16), activation='relu'))
    model.add(Conv2D(filters=10, kernel_size=(2,2), padding='same', input_shape=(8, 8, 16), activation='relu'))

    model.add(Flatten())
    model.add(Dense(2, activation='softmax'))
    adam = Adam(lr=1e-4, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
epochs = 500
n_batch = 8

model = cnn_model()
model.fit_generator(datagen.flow(X_train, y_train, batch_size=n_batch),
                    steps_per_epoch=len(X_train) / n_batch,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

train_score = model.evaluate(X_train, y_train, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)
print('Train Loss:{0:.3f}'.format(train_score[0]))
print('Train accuracy:{0:.3}'.format(train_score[1]))
print('Test Loss:{0:.3f}'.format(test_score[0]))
print('Test accuracy:{0:.3}'.format(test_score[1]))

ランダムな画像処理を行っているため、結果は毎回変わりますが、この文書を書いている実行の回では
> Train Loss:0.575
> Train accuracy:0.746
> Test Loss:0.605
> Test accuracy:0.68

という結果になりました。
Loss/accuracyともに、前回よりも悪い値になっているのですが、  
TrainとTestの値が近く、バランスよく学習できていることが伺えます。  
  
Trainの値がよく、Testの値が悪い状況は一般的に「過学習」や「汎化性能が低い」と呼ばれており  
本番データで良い結果が出せない傾向が強く、よくありません。

# 6. 学習済みモデルで判定　〜　7.提出用ファイルの作成

## 判定用ファイルの読み込み

In [ ]:
import glob
from pathlib import Path

test_images = None
test_filenames = None
for test_filepath in glob.glob('/kaggle/input/mj1-anomaly-images-detection-challenge/test/*.png'):
    tmp_image = load_preprocessed_image(test_filepath)
    if (test_images is None):
        test_images = tmp_image
        test_filenames = [Path(test_filepath).name]
    else:
        test_images = np.vstack((test_images, tmp_image))
        test_filenames.append(Path(test_filepath).name)

In [ ]:
result_predict = model.predict(test_images)
result_predict = np.argmax(result_predict, axis=1)
result_predict

In [ ]:
submit_filepath = "/kaggle/input/mj1-anomaly-images-detection-challenge/sample_submit.csv"
submit_df = pd.read_csv(submit_filepath, index_col=0)

for i, filename in enumerate(test_filenames):
    submit_df.loc[filename, 'Predicted'] = result_predict[i]
submit_df[:20]

In [ ]:
submit_df.to_csv('result_submit.csv')